- Remove non alphanumeric characters for simple training

In [1051]:
from transformer import Transformer # this is the transformer.py file
import torch
import numpy as np

In [1052]:
# 1) Fayl nomlarini belgilang
english_file = r"C:\Users\Maftuna\Desktop\English.txt"
turkish_file = r"C:\Users\Maftuna\Desktop\Turkish.txt"

In [1053]:
PADDING_TOKEN = ''
START_TOKEN = ''
END_TOKEN = ''


turkish_vocabulary = [
    START_TOKEN, ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/',
    '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
    ':', ';', '<', '=', '>', '?', '@',
    'A', 'B', 'C', 'Ç', 'D', 'E', 'F', 'G', 'Ğ', 'H', 'I', 'İ', 'J', 'K', 'L',
    'M', 'N', 'O', 'Ö', 'P', 'Q', 'R', 'S', 'Ş', 'T', 'U', 'Ü', 'V', 'W', 'X', 'Y', 'Z',
    '[', '\\', ']', '^', '_', '`',
    'a', 'b', 'c', 'ç', 'd', 'e', 'f', 'g', 'ğ', 'h', 'ı', 'i', 'j', 'k', 'l',
    'm', 'n', 'o', 'ö', 'p', 'q', 'r', 's', 'ş', 't', 'u', 'ü', 'v', 'w', 'x', 'y', 'z',
    '{', '|', '}', '~',
    '€', '£', '₺', '₽', '¥',  # pul birliklari
    PADDING_TOKEN, END_TOKEN
]

index_to_turkish = {k:v for k,v in enumerate(turkish_vocabulary)}
turkish_to_index = {v:k for k,v in enumerate(turkish_vocabulary)}




In [1054]:
START_TOKEN = ''
PADDING_TOKEN = ''
END_TOKEN = ''

english_vocabulary = [
    START_TOKEN, ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/',
    '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
    ':', ';', '<', '=', '>', '?', '@',
    'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L',
    'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z',
    '[', '\\', ']', '^', '_', '`',
    'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l',
    'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z',
    '{', '|', '}', '~',
    PADDING_TOKEN, END_TOKEN
]

In [1055]:

# Create mappings for Turkish vocabulary
index_to_turkish = {k: v for k, v in enumerate(turkish_vocabulary)}
turkish_to_index = {v: k for k, v in enumerate(turkish_vocabulary)}

# Create mappings for English vocabulary
index_to_english = {k: v for k, v in enumerate(english_vocabulary)}
english_to_index = {v: k for k, v in enumerate(english_vocabulary)}

In [1056]:
with open(turkish_file, 'r', encoding='utf-8') as file:
    turkish_sentences = file.readlines()
with open(english_file, 'r', encoding='utf-8') as file:
    english_sentences = file.readlines()

# Limit Number of sentences
TOTAL_SENTENCES = 5074
english_sentences = english_sentences[:TOTAL_SENTENCES]
turkish_sentences = turkish_sentences[:TOTAL_SENTENCES]
english_sentences = [sentence.rstrip('\n').lower() for sentence in english_sentences]
turkish_sentences = [sentence.rstrip('\n') for sentence in turkish_sentences]

In [1057]:

english_sentences[:8]

['the members of femen stripped their tops in a school in üsküdar (a conservative district of i̇stanbul) and shouted slogans on the table covered with voting cards.',
 'it turned out the polling station subjected to femen protest was where pm recep tayyip erdoğan would cast his vote in i̇stanbul.',
 '"femen activists stormed the voting papers where pm erdoğan would cast his vote.”',
 'previously, femen turkey protested against the twitter ban in turkey with a femen activist sharing her bare-breast photo with "#dns" and "#direntwitter” (resisttwitter) and dns numbers written on it. (ea/bd)',
 'photo credit: \u200f@vita__nova , twitter',
 "as gunshots resounded in hakkari province bağlar neighborhood and other locations, one person, named mustafa er died and another was injured, according to news on yüksekovahaber\xa0website. mustafa er's burial was held late last night (july 20) in bağlar neighborhood. 43-year-old er was reported to be a father of 9.",
 'ambulances carried four people i

In [1058]:
turkish_sentences[:8]

["FEMEN üyeleri Üsküdar'daki bir okulda soyunarak oy pusulalarının bulunduğu masaya çıkıp slogan attı. İki sekstivist sandık merkezinde görevli polisler tarafından gözaltına alındı.",
 "FEMEN aktivistleri Erdoğan'ın İstanbul'da oy kullanacağı yerde oy zarflarını fırlattı. #ErdoğanıYasaklayın pic.twitter.com/msuCqlQnAa",
 '"FEMEN aktivistleri Erdoğan\'ın İstanbul\'da oy kullanacağı yerde oy zarflarını fırlattı"',
 'FEMEN Türkiye, geçen hafta da Twitter’dan bir sekstivistin memeleri üzerinde "#DNS" ve "#DirenTwitter" hashtag\'leri ve DNS sayıları yazılı fotoğraf paylaşarak Türkiye\'deki twitter yasağını protesto etmişti. (EA)',
 'Fotoğraf: \u200f@Vita__Nova , Twitter',
 "Yüksekovahaber sitesinin haberine göre Hakkari'nin Bağlar mahallesi ile farklı noktalarda silah sesleri yükselirken kavgada Mustafa Er isimli bir kişi hayatını kaybetti, bir kişi de yaralandı. Mustafa Er'in cenazesinin bu akşam Bağlar Mahallesi'nde toprağa verilmesi bekleniyor. 43 yaşındaki Er'in 9 çocuk babası olduğu öğ

In [1059]:
import numpy as np
PERCENTILE = 97
print( f"{PERCENTILE}th percentile length Kannada: {np.percentile([len(x) for x in turkish_sentences], PERCENTILE)}" )
print( f"{PERCENTILE}th percentile length English: {np.percentile([len(x) for x in english_sentences], PERCENTILE)}" )


97th percentile length Kannada: 403.0
97th percentile length English: 367.619999999999


In [1060]:
max_sequence_length = 200

def is_valid_tokens(sentence, vocab):
    for token in list(set(sentence)):
        if token not in vocab:
            return False
    return True

def is_valid_length(sentence, max_sequence_length):
    return len(list(sentence)) < (max_sequence_length - 1) # need to re-add the end token so leaving 1 space

valid_sentence_indicies = []
for index in range(len(turkish_sentences)):
    turkish_sentence, english_sentence = turkish_sentences[index], english_sentences[index]
    if is_valid_length(turkish_sentence, max_sequence_length) \
      and is_valid_length(english_sentence, max_sequence_length) \
      and is_valid_tokens(turkish_sentence, turkish_vocabulary):
        valid_sentence_indicies.append(index)

print(f"Number of sentences: {len(turkish_sentences)}")
print(f"Number of valid sentences: {len(valid_sentence_indicies)}")

Number of sentences: 5074
Number of valid sentences: 2263


In [1061]:
turkish_sentences = [turkish_sentences[i] for i in valid_sentence_indicies]
english_sentences = [english_sentences[i] for i in valid_sentence_indicies]

In [1062]:
turkish_sentences[:6]

["FEMEN üyeleri Üsküdar'daki bir okulda soyunarak oy pusulalarının bulunduğu masaya çıkıp slogan attı. İki sekstivist sandık merkezinde görevli polisler tarafından gözaltına alındı.",
 "FEMEN aktivistleri Erdoğan'ın İstanbul'da oy kullanacağı yerde oy zarflarını fırlattı. #ErdoğanıYasaklayın pic.twitter.com/msuCqlQnAa",
 '"FEMEN aktivistleri Erdoğan\'ın İstanbul\'da oy kullanacağı yerde oy zarflarını fırlattı"',
 "Olaylarda farklı zamanlarda yaralanan dört kişi ise ambulanslar ile Van'a sevk edildi.",
 'Aşiret büyüklerinden itidal çağrısı',
 'Valilik: İkinci bir emre kadar sokağa çıkmak yasak']

In [1063]:

import torch

d_model = 512
batch_size = 30
ffn_hidden = 2048
num_heads = 8
drop_prob = 0.1
num_layers = 1
max_sequence_length = 200
tr_vocab_size = len(turkish_vocabulary)

transformer = Transformer(d_model, 
                          ffn_hidden,
                          num_heads, 
                          drop_prob, 
                          num_layers, 
                          max_sequence_length,
                          tr_vocab_size,
                          english_to_index,
                          turkish_to_index,
                          START_TOKEN, 
                          END_TOKEN, 
                          PADDING_TOKEN)

In [1064]:

transformer

Transformer(
  (encoder): Encoder(
    (sentence_embedding): SentenceEmbedding(
      (embedding): Embedding(96, 512)
      (position_encoder): PositionalEncoding()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (layers): SequentialEncoder(
      (0): EncoderLayer(
        (attention): MultiHeadAttention(
          (qkv_layer): Linear(in_features=512, out_features=1536, bias=True)
          (linear_layer): Linear(in_features=512, out_features=512, bias=True)
        )
        (norm1): LayerNormalization()
        (dropout1): Dropout(p=0.1, inplace=False)
        (ffn): PositionwiseFeedForward(
          (linear1): Linear(in_features=512, out_features=2048, bias=True)
          (linear2): Linear(in_features=2048, out_features=512, bias=True)
          (relu): ReLU()
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (norm2): LayerNormalization()
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (decoder): Decoder(
    (sentence_embedding):

In [1065]:
from torch.utils.data import Dataset, DataLoader

class TextDataset(Dataset):

    def __init__(self, english_sentences, turkish_sentences):
        self.english_sentences = english_sentences
        self.turkish_sentences = turkish_sentences

    def __len__(self):
        return len(self.english_sentences)

    def __getitem__(self, idx):
        return self.english_sentences[idx], self.turkish_sentences[idx]
     

In [1066]:

dataset = TextDataset(english_sentences, turkish_sentences)

In [1067]:

len(dataset)

2263

In [1068]:

dataset[1]

('it turned out the polling station subjected to femen protest was where pm recep tayyip erdoğan would cast his vote in i̇stanbul.',
 "FEMEN aktivistleri Erdoğan'ın İstanbul'da oy kullanacağı yerde oy zarflarını fırlattı. #ErdoğanıYasaklayın pic.twitter.com/msuCqlQnAa")

In [1069]:

train_loader = DataLoader(dataset, batch_size)
iterator = iter(train_loader)

In [1070]:
for batch_num, batch in enumerate(iterator):
    print(batch)
    if batch_num > 3:
        break

[('the members of femen stripped their tops in a school in üsküdar (a conservative district of i̇stanbul) and shouted slogans on the table covered with voting cards.', 'it turned out the polling station subjected to femen protest was where pm recep tayyip erdoğan would cast his vote in i̇stanbul.', '"femen activists stormed the voting papers where pm erdoğan would cast his vote.”', 'ambulances carried four people injured at different stages of the events to van province.', 'tribal elders call to equanimity', 'governorate: curfew until further notice', 'entering or exiting the town center has been forbidden. lots of vehicles with citizens wanting to reach the town center are being held at depin police checkpoint.', 'dbp provincial co-chair: provocation', 'democratic regions party (dbp) province co-chair m. miraz çallı made the declaration on the inter-tribal conflict:', 'the southeast (turabdin) assyrian culture and solidarity association opened the center as part of the project backed 

In [1071]:
# Define the padding token
PADDING_TOKEN = ''
START_TOKEN = ''
END_TOKEN = ''


# Ensure the padding token is included in the Turkish vocabulary
if PADDING_TOKEN not in turkish_vocabulary:
    turkish_vocabulary.append(PADDING_TOKEN)

# Create mappings for Turkish vocabulary
index_to_turkish = {k: v for k, v in enumerate(turkish_vocabulary)}
turkish_to_index = {v: k for k, v in enumerate(turkish_vocabulary)}

# Create mappings for English vocabulary
index_to_english = {k: v for k, v in enumerate(english_vocabulary)}
english_to_index = {v: k for k, v in enumerate(english_vocabulary)}

from torch import nn

# Define the criterion with the correct ignore_index
criterian = nn.CrossEntropyLoss(ignore_index=turkish_to_index[PADDING_TOKEN],
                                reduction='none')

# When computing the loss, we are ignoring cases when the label is the padding token
for params in transformer.parameters():
    if params.dim() > 1:
        nn.init.xavier_uniform_(params)

optim = torch.optim.Adam(transformer.parameters(), lr=1e-4)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [1072]:
NEG_INFTY = -1e9

def create_masks(eng_batch, tr_batch):
    num_sentences = len(eng_batch)
    look_ahead_mask = torch.full([max_sequence_length, max_sequence_length] , True)
    look_ahead_mask = torch.triu(look_ahead_mask, diagonal=1)
    encoder_padding_mask = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)
    decoder_padding_mask_self_attention = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)
    decoder_padding_mask_cross_attention = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)

    for idx in range(num_sentences):
      eng_sentence_length, tr_sentence_length = len(eng_batch[idx]), len(tr_batch[idx])
      eng_chars_to_padding_mask = np.arange(eng_sentence_length + 1, max_sequence_length)
      tr_chars_to_padding_mask = np.arange(tr_sentence_length + 1, max_sequence_length)
      encoder_padding_mask[idx, :, eng_chars_to_padding_mask] = True
      encoder_padding_mask[idx, eng_chars_to_padding_mask, :] = True
      decoder_padding_mask_self_attention[idx, :, tr_chars_to_padding_mask] = True
      decoder_padding_mask_self_attention[idx, tr_chars_to_padding_mask, :] = True
      decoder_padding_mask_cross_attention[idx, :, eng_chars_to_padding_mask] = True
      decoder_padding_mask_cross_attention[idx, tr_chars_to_padding_mask, :] = True

    encoder_self_attention_mask = torch.where(encoder_padding_mask, NEG_INFTY, 0)
    decoder_self_attention_mask =  torch.where(look_ahead_mask + decoder_padding_mask_self_attention, NEG_INFTY, 0)
    decoder_cross_attention_mask = torch.where(decoder_padding_mask_cross_attention, NEG_INFTY, 0)
    return encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask
     

In [1073]:
# Add the missing character to the vocabulary
missing_char = '’'
if missing_char not in turkish_vocabulary:
    turkish_vocabulary.append(missing_char)
    index_to_turkish[len(turkish_vocabulary) - 1] = missing_char
    turkish_to_index[missing_char] = len(turkish_vocabulary) - 1

if missing_char not in english_vocabulary:
    english_vocabulary.append(missing_char)
    index_to_english[len(english_vocabulary) - 1] = missing_char
    english_to_index[missing_char] = len(english_vocabulary) - 1

Modify mask such that the padding tokens cannot look ahead.
In Encoder, tokens before it should be -1e9 while tokens after it should be -inf.
 

Note the target mask starts with 2 rows of non masked items: https://github.com/SamLynnEvans/Transformer/blob/master/Beam.py#L55


In [1074]:
vocab_char = 'ü'
dataset_char = 'ü' # datasetdan kelayotgan belgini yozing yoki olib ko‘ring batch[1][sentence_num][x]

print("Vocab 'ü':", ord(vocab_char))
print("Dataset 'ü':", ord(dataset_char))
print(vocab_char == dataset_char)


Vocab 'ü': 252
Dataset 'ü': 252
True


In [1075]:
sentence = "örnek üğşı"
tokenized = [turkish_to_index[char] for char in sentence]
print(tokenized)


[90, 93, 88, 77, 85, 1, 98, 80, 95, 82]


In [1076]:
transformer = Transformer(
    tr_vocab_size=len(english_vocabulary),
    tr_vocab_size=len(turkish_vocabulary),
    src_pad_idx=english_to_index[PADDING_TOKEN],
    trg_pad_idx=turkish_to_index[PADDING_TOKEN],
    english_to_index=english_to_index,
    turkish_to_index=turkish_to_index,
).to(device)

SyntaxError: keyword argument repeated: tr_vocab_size (1107686551.py, line 3)

In [ ]:
transformer.train()
transformer.to(device)
total_loss = 0
num_epochs = 10

for epoch in range(num_epochs):
    print(f"Epoch {epoch}")
    iterator = iter(train_loader)
    for batch_num, batch in enumerate(iterator):
        transformer.train()
        eng_batch, tr_batch = batch
        encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask = create_masks(eng_batch, kn_batch)
        optim.zero_grad()
        tr_predictions = transformer(eng_batch,
                                     tr_batch,
                                     encoder_self_attention_mask.to(device), 
                                     decoder_self_attention_mask.to(device), 
                                     decoder_cross_attention_mask.to(device),
                                     enc_start_token=False,
                                     enc_end_token=False,
                                     dec_start_token=True,
                                     dec_end_token=True)
        labels = transformer.decoder.sentence_embedding.batch_tokenize(tr_batch, start_token=False, end_token=True)
        loss = criterian(
            tr_predictions.view(-1, tr_vocab_size).to(device),
            labels.view(-1).to(device)
        ).to(device)
        valid_indicies = torch.where(labels.view(-1) == turkish_to_index[PADDING_TOKEN], False, True)
        loss = loss.sum() / valid_indicies.sum()
        loss.backward()
        optim.step()
        #train_losses.append(loss.item())
        if batch_num % 100 == 0:
            print(f"Iteration {batch_num} : {loss.item()}")
            print(f"English: {eng_batch[0]}")
            print(f"Turkish Translation: {tr_batch[0]}")
            tr_sentence_predicted = torch.argmax(tr_predictions[0], axis=1)
            predicted_sentence = ""
            for idx in tr_sentence_predicted:
              if idx == turkish_to_index[END_TOKEN]:
                break
              predicted_sentence += index_to_turkish[idx.item()]
            print(f"Turkish Prediction: {predicted_sentence}")


            transformer.eval()
            tr_sentence = ("",)
            eng_sentence = ("should we go to the mall?",)
            for word_counter in range(max_sequence_length):
                encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask= create_masks(eng_sentence, kn_sentence)
                predictions = transformer(eng_sentence,
                                          tr_sentence,
                                          encoder_self_attention_mask.to(device), 
                                          decoder_self_attention_mask.to(device), 
                                          decoder_cross_attention_mask.to(device),
                                          enc_start_token=False,
                                          enc_end_token=False,
                                          dec_start_token=True,
                                          dec_end_token=False)
                next_token_prob_distribution = predictions[0][word_counter] # not actual probs
                next_token_index = torch.argmax(next_token_prob_distribution).item()
                next_token = index_to_turkish[next_token_index]
                tr_sentence = (tr_sentence[0] + next_token, )
                if next_token == END_TOKEN:
                  break
            
            print(f"Evaluation translation (should we go to the mall?) : {tr_sentence}")
            print("-------------------------------------------")
     

Epoch 0


KeyError: 'ü'

## Inference

In [ ]:
transformer.eval()
def translate(eng_sentence):
  eng_sentence = (eng_sentence,)
  kn_sentence = ("",)
  for word_counter in range(max_sequence_length):
    encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask= create_masks(eng_sentence, kn_sentence)
    predictions = transformer(eng_sentence,
                              kn_sentence,
                              encoder_self_attention_mask.to(device), 
                              decoder_self_attention_mask.to(device), 
                              decoder_cross_attention_mask.to(device),
                              enc_start_token=False,
                              enc_end_token=False,
                              dec_start_token=True,
                              dec_end_token=False)
    next_token_prob_distribution = predictions[0][word_counter]
    next_token_index = torch.argmax(next_token_prob_distribution).item()
    next_token = index_to_turkish[next_token_index]
    kn_sentence = (kn_sentence[0] + next_token, )
    if next_token == END_TOKEN:
      break
  return kn_sentence[0]

In [ ]:
translation = translate("what should we do when the day starts?")
print(translation)
#ದಿನ ಪ್ರಾರಂಭವಾದಾಗ ನಾವು ಏನು ಮಾಡಬೇಕು?

00000çççç00000ç:::0000::::000000000000000000000000000000000000000000000000000000ç30000000000000000000000000000000:::::::00000000000000000000çç:::::0000000:::::0000::::::::::00000000000000000000000ç3ee


In [ ]:
translation = translate("how is this the truth?")
print(translation)
#ಇದು ಹೇಗೆ ಸತ್ಯ

ಹೇಗೆ ಇದು ಹೇಗೆ ಹೇಗೆ?<END>


In [ ]:
translation = translate("the world is a large place with different people")
print(translation)
#ಪ್ರಪಂಚವು ವಿಭಿನ್ನ ಜನರೊಂದಿಗೆ ದೊಡ್ಡ ಸ್ಥಳವಾಗಿದೆ

ಇದರಿಂದ ಮೂಲಕ ಸಂಬಂಧಿಸಿದ ಮೇಲೆ ಮಾಡಿದ್ದಾರೆ<END>


In [ ]:
translation = translate("my name is ajay")
print(translation)
#ನನ್ನ ಹೆಸರು ಅಜಯ್

ನಾನು ಕುಟುಂಬದ ಹೆಸರು<END>


In [ ]:
translation = translate("i cannot stand this smell")
print(translation)
#ನಾನು ಈ ವಾಸನೆಯನ್ನು ಸಹಿಸುವುದಿಲ್ಲ

ನಾನು ಅಂತರ ಸಂಗತಿ ನಾನು ಕೊಡುವುದಿಲ್ಲ<END>


In [ ]:
translation = translate("noodles are the best")
print(translation)

ಯಾವುದೇ ಕಾರಣಗಳು ಇಲ್ಲ<END>


In [ ]:
translation = translate("why care about this?")
print(translation)

IndexError: index out of range in self

This translated pretty well : "What is the reason. Why" without punctuation.

In [ ]:
translation = translate("this is the best thing ever")
print(translation)
# ಇದು ಎಂದೆಂದಿಗೂ ಉತ್ತಮವಾಗಿದೆ

ಇದು ಅತ್ಯಂತ ಹೊರತಾಗಿದೆ<END>


The translation : "This is very unusual"

In [ ]:
translation = translate("i am here")
print(translation)
# ನಾನು ಇಲ್ಲಿದ್ದೇನೆ

ನಾನು ಕೇಳಿದ್ದೇನೆ.<END>


Translation: "I have heard". 
This is why word based translator may perform better than character translator. This is actually very good at optimizing the objective of the current transformer even though the translation is off.

In [ ]:
translation = translate("click this")
print(translation)
# ಇದನ್ನು ಕ್ಲಿಕ್ ಮಾಡಿ

ಕ್ಯಾನ್ ಕ್ಲಿಕ್ ಕ್ಲಿಕ್ ಕ್ಲಿಕ್ ಕ್ಲಿಕ್ ಕ್ಲಿಕ್ ಮಾಡಿ<END>


In [ ]:
translation = translate("where is the mall?")
print(translation)

ಎಲ್ಲಿ ಎಲ್ಲಿ ಎಲ್ಲಿ?<END>


In [ ]:
translation = translate("what should we do?")
print(translation)

ಏನು ಮಾಡಬೇಕು?<END>


This is correct; but it absolutely fumbles on the next one

In [ ]:
translation = translate("today, what should we do")
print(translation)

ಅದನ್ನು ಮಾಡಿದ ಮೇಲೆ ಮಾಡಿದ ಮಾಡಿದ್ದಾರೆ<END>


In [ ]:
translation = translate("why did they activate?")
print(translation)
# ಅವರು ಏಕೆ ಸಕ್ರಿಯಗೊಳಿಸಿದರು?

ಅದರ ಮೇಲೆ ಏಕೆ ಮಾಡಿದ್ದಾರೆ?<END>


In [ ]:
translation = translate("why did they do this?")
print(translation)
# ಅವರು ಇದನ್ನು ಏಕೆ ಮಾಡಿದರು?

ಅವರು ಏಕೆ ಅವರು ಏಕೆ ಮಾಡಿದ್ದಾರೆ?<END>


That turned out well!

In [ ]:
translation = translate("i am well.")
print(translation)
# ನಾನು ಆರಾಮವಾಗಿದ್ದೇನೆ

ನಾನು ನಾನು ಕೊಡುತ್ತೇನೆ.<END>


Translation: "I will give you something"

In [ ]:
translation = translate("whats the word on the street?")
print(translation)

ಇದರ ಬಗ್ಗೆ ಏನು?<END>


Kind of close semantically. Translation is something like: "What is this about"

## Insights

- When training, we can treat every alphabet as a single unit instead of splitting it into it's corresponding parts to preserve meaning. For example, ಮಾ should be 1 unit when comuting a loss. It should not be decomposed into ಮ + ఆ
- Using word-based or BPE based tokenizations may help mitigate (1). Also, we will get valid word (or BPE) units if we do so. 
- Make sure the training set has a large variety of sentences that are not just about one topic like "work" and "government"
- Increase the number of encoder / decoder units for better translations. It was set to the minimum of 1 of each unit here.
- Create a translator with a language you understand ideally.

Overall, this model definately learned something. And you can use other languages instead of this kannada language and might see better luck